In [5]:
import numpy as np
import pandas as pd
import nibabel as nib
from nilearn import input_data, image

### Pre-Modafinil

Define paths to input data (taken from an `fmriprep` run on OpenNeuro).

In [19]:
bold_T1w_preproc = '/Users/dlurie/Dropbox/Projects/avalanche/avalanche/data/ds000133/sub-01/pre/sub-01_ses-pre_task-rest_run-01_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'

In [20]:
bold_T1w_brainmask = '/Users/dlurie/Dropbox/Projects/avalanche/avalanche/data/ds000133/sub-01/pre/sub-01_ses-pre_task-rest_run-01_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz'

In [8]:
bold_confounds = '/Users/dlurie/Dropbox/Projects/avalanche/avalanche/data/ds000133/sub-01/pre/sub-01_ses-pre_task-rest_run-01_bold_confounds.tsv'

Load confounds.

In [9]:
all_confounds = pd.read_csv(bold_confounds, sep='\t')

In [10]:
all_confounds.columns

Index(['WhiteMatter', 'GlobalSignal', 'stdDVARS', 'non-stdDVARS',
       'vx-wisestdDVARS', 'FramewiseDisplacement', 'tCompCor00', 'tCompCor01',
       'tCompCor02', 'tCompCor03', 'tCompCor04', 'tCompCor05', 'aCompCor00',
       'aCompCor01', 'aCompCor02', 'aCompCor03', 'aCompCor04', 'aCompCor05',
       'Cosine00', 'Cosine01', 'NonSteadyStateOutlier00', 'X', 'Y', 'Z',
       'RotX', 'RotY', 'RotZ'],
      dtype='object')

In [12]:
nuisance_vars = all_confounds[['aCompCor00', 'aCompCor01', 'aCompCor02', 'aCompCor03', 'aCompCor04', 'aCompCor05',
                              'X', 'Y', 'Z', 'RotX', 'RotY', 'RotZ']].copy()

Calculate Friston-24 (Yan et al. variant) Motion Parameters

In [14]:
# Create squared versions of each motion parameter at Tt
for mopar in ['X', 'Y', 'Z', 'RotX', 'RotY', 'RotZ']:
    sq_col_name = mopar+'sq'
    sq_col_data = np.square(nuisance_vars[mopar])
    nuisance_vars.loc[:,sq_col_name] = sq_col_data

In [15]:
zpad = pd.Series([0])
for mopar in nuisance_vars.columns[-12:]:
    pv_col_name = mopar+'_t-1'
    pv_col_data = zpad.append(nuisance_vars[mopar], ignore_index=True)[:-1]
    nuisance_vars.loc[:,pv_col_name] = pv_col_data

In [16]:
nuisance_vars.columns

Index(['aCompCor00', 'aCompCor01', 'aCompCor02', 'aCompCor03', 'aCompCor04',
       'aCompCor05', 'X', 'Y', 'Z', 'RotX', 'RotY', 'RotZ', 'Xsq', 'Ysq',
       'Zsq', 'RotXsq', 'RotYsq', 'RotZsq', 'X_t-1', 'Y_t-1', 'Z_t-1',
       'RotX_t-1', 'RotY_t-1', 'RotZ_t-1', 'Xsq_t-1', 'Ysq_t-1', 'Zsq_t-1',
       'RotXsq_t-1', 'RotYsq_t-1', 'RotZsq_t-1'],
      dtype='object')

Load EPI image and brain mask.

In [21]:
epi_img = image.load_img(bold_T1w_preproc)

In [22]:
mask_img = image.load_img(bold_T1w_brainmask)

Clean EPI image (do not standardize values)

In [41]:
clean_img = image.clean_img(epi_img, detrend=True, standardize=False, confounds=nuisance_vars.as_matrix(), low_pass=0.1, high_pass=0.01, t_r=1.671)

In [42]:
clean_img.to_filename('/Users/dlurie/Dropbox/Projects/avalanche/avalanche/data/ds000133/sub-01/pre/sub-01_ses-pre_task-rest_run-01_bold_space-MNI152NLin2009cAsym_clean.nii.gz')

Clean EPI image (standardize values)

In [39]:
clean_img_std = image.clean_img(epi_img, detrend=True, standardize=True, confounds=nuisance_vars.as_matrix(), low_pass=0.1, high_pass=0.01, t_r=1.671)

In [40]:
clean_img_std.to_filename('/Users/dlurie/Dropbox/Projects/avalanche/avalanche/data/ds000133/sub-01/pre/sub-01_ses-pre_task-rest_run-01_bold_space-MNI152NLin2009cAsym_clean_std.nii.gz')

Smooth standardized image

In [47]:
clean_img_std_fwhm6 = image.smooth_img(clean_img_std, 6)

In [48]:
clean_img_std_fwhm6.to_filename('/Users/dlurie/Dropbox/Projects/avalanche/avalanche/data/ds000133/sub-01/pre/sub-01_ses-pre_task-rest_run-01_bold_space-MNI152NLin2009cAsym_clean_std_fwhm6.nii.gz')

### Post-Modafinil

Define paths to input data (taken from an `fmriprep` run on OpenNeuro).

In [1]:
bold_T1w_preproc = '/Users/dlurie/Dropbox/Projects/avalanche/avalanche/data/ds000133/sub-01/post/sub-01_ses-post_task-rest_run-01_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'

In [2]:
bold_T1w_brainmask = '/Users/dlurie/Dropbox/Projects/avalanche/avalanche/data/ds000133/sub-01/post/sub-01_ses-post_task-rest_run-01_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz'

In [3]:
bold_confounds = '/Users/dlurie/Dropbox/Projects/avalanche/avalanche/data/ds000133/sub-01/post/sub-01_ses-post_task-rest_run-01_bold_confounds.tsv'

Load confounds.

In [6]:
all_confounds = pd.read_csv(bold_confounds, sep='\t')

In [7]:
all_confounds.columns

Index(['WhiteMatter', 'GlobalSignal', 'stdDVARS', 'non-stdDVARS',
       'vx-wisestdDVARS', 'FramewiseDisplacement', 'tCompCor00', 'tCompCor01',
       'tCompCor02', 'tCompCor03', 'tCompCor04', 'tCompCor05', 'aCompCor00',
       'aCompCor01', 'aCompCor02', 'aCompCor03', 'aCompCor04', 'aCompCor05',
       'Cosine00', 'Cosine01', 'X', 'Y', 'Z', 'RotX', 'RotY', 'RotZ'],
      dtype='object')

In [8]:
nuisance_vars = all_confounds[['aCompCor00', 'aCompCor01', 'aCompCor02', 'aCompCor03', 'aCompCor04', 'aCompCor05',
                              'X', 'Y', 'Z', 'RotX', 'RotY', 'RotZ']].copy()

Calculate Friston-24 (Yan et al. variant) Motion Parameters

In [9]:
# Create squared versions of each motion parameter at Tt
for mopar in ['X', 'Y', 'Z', 'RotX', 'RotY', 'RotZ']:
    sq_col_name = mopar+'sq'
    sq_col_data = np.square(nuisance_vars[mopar])
    nuisance_vars.loc[:,sq_col_name] = sq_col_data

In [10]:
zpad = pd.Series([0])
for mopar in nuisance_vars.columns[-12:]:
    pv_col_name = mopar+'_t-1'
    pv_col_data = zpad.append(nuisance_vars[mopar], ignore_index=True)[:-1]
    nuisance_vars.loc[:,pv_col_name] = pv_col_data

In [11]:
nuisance_vars.columns

Index(['aCompCor00', 'aCompCor01', 'aCompCor02', 'aCompCor03', 'aCompCor04',
       'aCompCor05', 'X', 'Y', 'Z', 'RotX', 'RotY', 'RotZ', 'Xsq', 'Ysq',
       'Zsq', 'RotXsq', 'RotYsq', 'RotZsq', 'X_t-1', 'Y_t-1', 'Z_t-1',
       'RotX_t-1', 'RotY_t-1', 'RotZ_t-1', 'Xsq_t-1', 'Ysq_t-1', 'Zsq_t-1',
       'RotXsq_t-1', 'RotYsq_t-1', 'RotZsq_t-1'],
      dtype='object')

Load EPI image and brain mask.

In [12]:
epi_img = image.load_img(bold_T1w_preproc)

In [13]:
mask_img = image.load_img(bold_T1w_brainmask)

Clean EPI image (do not standardize values)

In [14]:
clean_img = image.clean_img(epi_img, detrend=True, standardize=False, confounds=nuisance_vars.as_matrix(), low_pass=0.1, high_pass=0.01, t_r=1.671)

In [15]:
clean_img.to_filename('/Users/dlurie/Dropbox/Projects/avalanche/avalanche/data/ds000133/sub-01/post/sub-01_ses-post_task-rest_run-01_bold_space-MNI152NLin2009cAsym_clean.nii.gz')

Clean EPI image (standardize values)

In [16]:
clean_img_std = image.clean_img(epi_img, detrend=True, standardize=True, confounds=nuisance_vars.as_matrix(), low_pass=0.1, high_pass=0.01, t_r=1.671)

In [17]:
clean_img_std.to_filename('/Users/dlurie/Dropbox/Projects/avalanche/avalanche/data/ds000133/sub-01/post/sub-01_ses-post_task-rest_run-01_bold_space-MNI152NLin2009cAsym_clean_std.nii.gz')

Smooth standardized image

In [18]:
clean_img_std_fwhm6 = image.smooth_img(clean_img_std, 6)

In [19]:
clean_img_std_fwhm6.to_filename('/Users/dlurie/Dropbox/Projects/avalanche/avalanche/data/ds000133/sub-01/post/sub-01_ses-post_task-rest_run-01_bold_space-MNI152NLin2009cAsym_clean_std_fwhm6.nii.gz')